In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file = open("/content/drive/MyDrive/rugare.txt","r")
text = file.read()

In [5]:
text

'Zita rekuberekwa ndinonzi Rugare, ndiri mwana wechi6 mumhuri medu takaberekwa tiri 8 vakomana vatatu nevasikana 5. Kuvasikana ndiri wechi4. Ndine makore\t18 ekuberekwa. Chikoro ndakagumira hangu paOlevel yandakapasawo zvekubatanidzira asi handishori hangu Mwari.\n\n\nMushure mekupedza 0 level yangu ndakambogara pamba pevabereki vangu ndichitoshaya kuti ndotangira papi.\nMumusoro mangu mabasa andaifunga kuti akanaka ndiyo nursing neteaching saka ndaingotiwo ndichazoedzawo hangu\n \nnursing sezvo teaching ndaitoityawo.\n\nKuchurch ndaienda chaizvo ende Mwari ndaimuda zvakanyanya sezvo ndakakurira mumhuri yainamatwa.\n\nNdiripo pamba pevabereki vangu kudaro ndayiita mabasa ese anoitwa kumusha.\nTaigara nemasisters angu maviri nekahanzvadzi kaiva kagotwe nababa naamai. Vamwe vakanga vakura vava nemhuri dzavo.\n\nRimwe zuva tichibva hedu kuchurch iri Sunday ndakawana pamba pedu pane cousin sister yedu achiti anga awana munhu ayida musikana webasa\n \nwemumba. Nekuda kwekuti ndakanga ndakus

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
encoded_data = tokenizer.texts_to_sequences([text])[0]
vocabulary_size = len(tokenizer.word_index) + 1

In [8]:
# Saving the tokenizer object for future use
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
sequences = []

for i in range(1, len(encoded_data)):
    sequence = encoded_data[:i+1]
    sequences.append(sequence)

sequences = np.array(pad_sequences(sequences, maxlen=6, padding='pre'))
X, y = sequences[:, :-1], sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=vocabulary_size)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from gensim.models import Word2Vec

In [12]:
# Creating word embeddings
sentences = [sentence.split() for sentence in text.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")

In [13]:
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:

        pass

In [18]:
first_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocabulary_size, 100, input_length=5),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.LSTM(100),
  tf.keras.layers.Dense(vocabulary_size, activation='softmax')
])

first_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Training the first model
first_history = first_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
52/52 [==============================] - 6s 39ms/step - loss: 7.1397 - accuracy: 0.0236 - val_loss: 7.1428 - val_accuracy: 0.0314
Epoch 2/100
52/52 [==============================] - 2s 37ms/step - loss: 6.8773 - accuracy: 0.0254 - val_loss: 7.4321 - val_accuracy: 0.0266
Epoch 3/100
52/52 [==============================] - 2s 33ms/step - loss: 6.5728 - accuracy: 0.0254 - val_loss: 7.8139 - val_accuracy: 0.0266
Epoch 4/100
52/52 [==============================] - 1s 24ms/step - loss: 6.3629 - accuracy: 0.0254 - val_loss: 7.9965 - val_accuracy: 0.0242
Epoch 5/100
52/52 [==============================] - 1s 23ms/step - loss: 6.1825 - accuracy: 0.0254 - val_loss: 8.1026 - val_accuracy: 0.0290
Epoch 6/100
52/52 [==============================] - 1s 23ms/step - loss: 6.0097 - accuracy: 0.0254 - val_loss: 8.5571 - val_accuracy: 0.0217
Epoch 7/100
52/52 [==============================] - 1s 23ms/step - loss: 5.8236 - accuracy: 0.0248 - val_loss: 8.7730 - val_accuracy: 0.0217
Epoch 

In [16]:
second_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocabulary_size, 100, weights=[embedding_matrix], input_length=5, trainable=False),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.LSTM(100),
  tf.keras.layers.Dense(vocabulary_size, activation='softmax')
])

second_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Training the second model
second_history = second_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
52/52 [==============================] - 6s 37ms/step - loss: 7.1347 - accuracy: 0.0205 - val_loss: 7.1536 - val_accuracy: 0.0314
Epoch 2/100
52/52 [==============================] - 1s 21ms/step - loss: 6.9095 - accuracy: 0.0254 - val_loss: 7.3761 - val_accuracy: 0.0314
Epoch 3/100
52/52 [==============================] - 1s 22ms/step - loss: 6.7512 - accuracy: 0.0254 - val_loss: 7.9761 - val_accuracy: 0.0314
Epoch 4/100
52/52 [==============================] - 1s 21ms/step - loss: 6.7044 - accuracy: 0.0254 - val_loss: 8.2230 - val_accuracy: 0.0314
Epoch 5/100
52/52 [==============================] - 1s 21ms/step - loss: 6.6889 - accuracy: 0.0254 - val_loss: 8.2178 - val_accuracy: 0.0314
Epoch 6/100
52/52 [==============================] - 1s 21ms/step - loss: 6.6791 - accuracy: 0.0254 - val_loss: 8.2948 - val_accuracy: 0.0314
Epoch 7/100
52/52 [==============================] - 1s 21ms/step - loss: 6.6730 - accuracy: 0.0254 - val_loss: 8.3920 - val_accuracy: 0.0314
Epoch 

In [20]:
val_loss_first_model = first_history.history['val_loss'][-1]
val_loss_second_model = second_history.history['val_loss'][-1]

print(f"Validation Loss - First Model: {val_loss_first_model}")
print(f"Validation Loss - Second Model: {val_loss_second_model}")

Validation Loss - First Model: 14.19434928894043
Validation Loss - Second Model: 14.171158790588379


In [22]:
import matplotlib.pyplot as plt

In [21]:
def visualize(history):
    epoch_range = range (10)
    plt.figure(figsize=(10, 10))

    plt.subplot(1, 2, 1)
    plt.title('Training and Validation loss')
    training_loss = history.history.get('loss')
    validation_loss = history.history.get('val_loss')
    plt.plot(epoch_range, training_loss, label='Training loss')
    plt.plot(epoch_range, validation_loss, label='Validation loss')
    plt.legend(loc='lower left')

    plt.subplot(1, 2, 2)
    plt.title('Training and Validation accuracy')
    training_accuracy = history.history.get('accuracy')
    validation_accuracy = history.history.get('val_accuracy')
    plt.plot(epoch_range, training_accuracy, label='Training accuracy')
    plt.plot(epoch_range, validation_accuracy, label='Validation accuracy')
    plt.legend(loc='lower left')

    plt.show()

# Choose Second Model

In [27]:
second_model.save("second_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
# Load the previously saved model
model =  tf.keras.models.load_model('second_model.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    for _ in range(num_words):

        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')


        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)


        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break


        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Ask the user for input
user_input = input("Input 5 Shona Words: ")


predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")

Input 5 Shona Words: mamukasei baba naamai marara sei
The next words might be: ndakagashirwa mhosva ruru
